In [10]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt comet_ml  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-304-g22361691 Python-3.10.12 torch-2.0.1+cu117 CPU


Setup complete ✅ (4 CPUs, 3.7 GB RAM, 53.2/249.4 GB disk)


/home/mhsn/Files/PFA


In [13]:
%cd /home/mhsn/Files/PFA/yolov5/yolov5

/home/mhsn/Files/PFA/yolov5/yolov5


# Fixing data


In [ ]:
import os, shutil, random

# preparing the folder structure

full_data_path = 'data/obj/'
extension_allowed = '.jpg'
split_percentage = 90

images_path = 'data/images/'
if os.path.exists(images_path):
    shutil.rmtree(images_path)
os.mkdir(images_path)
    
labels_path = 'data/labels/'
if os.path.exists(labels_path):
    shutil.rmtree(labels_path)
os.mkdir(labels_path)
    
training_images_path = images_path + 'training/'
validation_images_path = images_path + 'validation/'
training_labels_path = labels_path + 'training/'
validation_labels_path = labels_path +'validation/'
    
os.mkdir(training_images_path)
os.mkdir(validation_images_path)
os.mkdir(training_labels_path)
os.mkdir(validation_labels_path)

files = []

ext_len = len(extension_allowed)

for r, d, f in os.walk(full_data_path):
    for file in f:
        if file.endswith(extension_allowed):
            strip = file[0:len(file) - ext_len]      
            files.append(strip)

random.shuffle(files)

size = len(files)                   

split = int(split_percentage * size / 100)

print("copying training data")
for i in range(split):
    strip = files[i]
                         
    image_file = strip + extension_allowed
    src_image = full_data_path + image_file
    shutil.copy(src_image, training_images_path) 
                         
    annotation_file = strip + '.txt'
    src_label = full_data_path + annotation_file
    shutil.copy(src_label, training_labels_path) 

print("copying validation data")
for i in range(split, size):
    strip = files[i]
                         
    image_file = strip + extension_allowed
    src_image = full_data_path + image_file
    shutil.copy(src_image, validation_images_path) 
                         
    annotation_file = strip + '.txt'
    src_label = full_data_path + annotation_file
    shutil.copy(src_label, validation_labels_path) 

print("finished")

# Training the model

In [17]:
python3 train.py --img 243 --epochs 30 --data dataset.yaml --weights yolov5s.pt --cache

SyntaxError: invalid syntax (1353071617.py, line 1)

# Loading the model

In [1]:
import torch

In [3]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/home/mhsn/Files/PFA/yolov5/best.pt', force_reload=True)


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /home/mhsn/.cache/torch/hub/master.zip
YOLOv5 🚀 2024-5-12 Python-3.10.12 torch-2.0.1+cu117 CPU

Fusing layers... 
[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.
Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


# Testing the output

In [ ]:
import cv2
import matplotlib.pyplot as plt
img=cv2.imread('/home/mhsn/Files/PFA/Dataset/Train/10dt/yoPJx7ljbZUAFoOpvrTWkfgQk7f.jpg')
cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
# Load the image
image = cv2.imread('/home/mhsn/Files/PFA/Dataset/Train/10dt/yoPJx7ljbZUAFoOpvrTWkfgQk7f.jpg')
#getting the results
result=model(image)
predictions = result.pred[0]
filtered_predictions = [pred for pred in predictions if pred[4] >= 0.7]
for pred in filtered_predictions:
    # Extract bounding box coordinates from the prediction
    x1, y1, x2, y2 = map(int, pred[:4].tolist())

    # Extract class index from the prediction
    class_index = int(pred[-1].item())

    # Retrieve the corresponding value from the mapping dictionary
    value = class_value_mapping.get(class_index)

    # Draw bounding box
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Display class label and value
    label = f"Class: {class_index}, Value: {value} dt"
    cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

print(x1,x2,y1,y2)

In [ ]:
import cv2
import torch
import matplotlib.pyplot as plt

# Load the image
image = cv2.imread('/home/mhsn/Files/PFA/Dataset/Train/10dt/yoPJx7ljbZUAFoOpvrTWkfgQk7f.jpg')
# Get results
result=model(image)

# Assuming result.pred[0] contains the tensor you provided
predictions = result.pred[0]

# Define the mapping dictionary
class_value_mapping = {
    0: 10,  # Class 0 corresponds to 10dt
    1: 20,  # Class 1 corresponds to 20dt
    2: 50,  # Class 2 corresponds to 50dt
    3: 5    # Class 3 corresponds to 5dt
}

# Filter predictions with confidence below 0.7
filtered_predictions = [pred for pred in predictions if pred[4] >= 0.7]

# Iterate through each filtered prediction
for pred in filtered_predictions:
    # Extract bounding box coordinates from the prediction
    x1, y1, x2, y2 = map(int, pred[:4].tolist())

    # Extract class index from the prediction
    class_index = int(pred[-1].item())

    # Retrieve the corresponding value from the mapping dictionary
    value = class_value_mapping.get(class_index)

    # Draw bounding box
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Display class label and value
    label = f"Class: {class_index}, Value: {value} dt"
    cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Convert image from BGR to RGB (required for displaying with plt.show)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Display the image using plt.show
plt.imshow(image_rgb)
plt.axis('off')  # Turn off axis
plt.show()
print(value)


In [ ]:
# Load the image
image = cv2.imread('/home/mhsn/Files/PFA/Dataset/Train/10dt/yoPJx7ljbZUAFoOpvrTWkfgQk7f.jpg')
#getting the results
result=model(image)

# Assuming result.pred[0] contains the tensor you provided
predictions = result.pred[0]

# testing on an image


In [ ]:
import cv2
import torch

# Define the mapping dictionary
class_value_mapping = {
    0: 10,  # Class 0 corresponds to 10dt
    1: 20,  # Class 1 corresponds to 20dt
    2: 50,  # Class 2 corresponds to 50dt
    3: 5    # Class 3 corresponds to 5dt
}

# Load the image
image = cv2.imread('/home/mhsn/Files/PFA/Dataset/Train/10dt/yoPJx7ljbZUAFoOpvrTWkfgQk7f.jpg')
#getting the results
result=model(image)

# Assuming result.pred[0] contains the tensor you provided
predictions = result.pred[0]

# Iterate through each prediction
for pred in predictions:
    # Extract confidence score from the prediction
    confidence = pred[4].item()

    # Only process predictions with confidence over 0.6
    if confidence > 0.6:
        # Extract bounding box coordinates from the prediction
        x1, y1, x2, y2 = map(int, pred[:4].tolist())

        # Extract class index from the prediction
        class_index = int(pred[-1].item())

        # Retrieve the corresponding value from the mapping dictionary
        value = class_value_mapping.get(class_index)

        # Draw bounding box
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Display class label and value
        label = f"Class: {class_index}, Value: {value} dt"
        cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Show the image
cv2.imshow("Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


# working on real time

In [3]:
import cv2
import torch

# Define the mapping dictionary
class_value_mapping = {
    0: 10,  # Class 0 corresponds to 10dt
    1: 20,  # Class 1 corresponds to 20dt
    2: 50,  # Class 2 corresponds to 50dt
    3: 5    # Class 3 corresponds to 5dt
}

# Load the pre-trained model
# Assuming model is already loaded and ready for inference

# Open a video capture device (webcam or video file)
cap = cv2.VideoCapture(0)  # Change to 0 for webcam, or provide video file path

# Check if the capture device is opened successfully
if not cap.isOpened():
    print("Error: Failed to open video capture device.")
    exit()

# Loop for real-time processing
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Check if the frame is empty (end of video)
    if not ret:
        print("Error: Failed to capture frame.")
        break

    # Perform inference on the frame
    result = model(frame)

    # Assuming result.pred[0] contains the tensor you provided
    predictions = result.pred[0]

    # Iterate through each prediction
    for pred in predictions:
        # Extract confidence score from the prediction
        confidence = pred[4].item()

        # Only process predictions with confidence over 0.6
        if confidence > 0.6:
            # Extract bounding box coordinates from the prediction
            x1, y1, x2, y2 = map(int, pred[:4].tolist())

            # Extract class index from the prediction
            class_index = int(pred[-1].item())

            # Retrieve the corresponding value from the mapping dictionary
            value = class_value_mapping.get(class_index)

            # Draw bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Display class label and value
            label = f"Class: {class_index}, Value: {value} dt"
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Frame', frame)

    # Check for 'q' key press to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture device and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


WARNING ⚠️ NMS time limit 0.550s exceeded


In [11]:
import os
import stat

# Define the output directory
output_directory = "/home/mhsn/Files/PFA/Output"

# Check if the directory exists
if not os.path.exists(output_directory):
    print("Error: The specified directory does not exist.")
    exit()

# Check if the directory has write permission
if not os.access(output_directory, os.W_OK):
    print("Error: The directory does not have write permission.")

    # Try to add write permission
    try:
        os.chmod(output_directory, stat.S_IWUSR | stat.S_IWGRP | stat.S_IWOTH)
        print("Write permission added to the directory.")
    except Exception as e:
        print(f"Error: Failed to add write permission to the directory: {e}")
        exit()
else:
    print("The directory has write permission.")


The directory has write permission.


# RUN THIS


In [3]:
 def say(text):
    
    import pyttsx3
    engine = pyttsx3.init()
    engine.setProperty('rate', 150)
    v=engine.getProperty('voices')
    
    engine.setProperty('voice',v[11].id )
    
    engine.say(text)
    engine.runAndWait()
    return 
        

In [5]:
import cv2
import torch
import os
output_file = os.path.join("/home/mhsn/Files/PFA/Output", "detected_frame.jpg")
i
# Define the mapping dictionary
class_value_mapping = {
    0: 10,  # Class 0 corresponds to 10dt
    1: 20,  # Class 1 corresponds to 20dt
    2: 50,  # Class 2 corresponds to 50dt
    3: 5    # Class 3 corresponds to 5dt
}

# Load the pre-trained model
# Assuming model is already loaded and ready for inference

# Open a video capture device (webcam or video file)
cap = cv2.VideoCapture(0)  # Change to 0 for webcam, or provide video file path

# Check if the capture device is opened successfully
if not cap.isOpened():
    print("Error: Failed to open video capture device.")
    exit()

# Loop for real-time processing
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Check if the frame is empty (end of video)
    if not ret:
        print("Error: Failed to capture frame.")
        break

    # Perform inference on the frame
    result = model(frame)

    # Assuming result.pred[0] contains the tensor you provided
    predictions = result.pred[0]

    # Flag to check if any detection occurred in the frame
    detection_occurred = False

    # Iterate through each prediction
    for pred in predictions:
        # Extract confidence score from the prediction
        confidence = pred[4].item()

        # Only process predictions with confidence over 0.6
        if confidence > 0.3:
            # Extract bounding box coordinates from the prediction
            x1, y1, x2, y2 = map(int, pred[:4].tolist())

            # Extract class index from the prediction
            class_index = int(pred[-1].item())

            # Retrieve the corresponding value from the mapping dictionary
            value = class_value_mapping.get(class_index)

            # Draw bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Display class label and value
            label = f"Class: {class_index}, Value: {value} dt, Confidance: {confidence}"
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Set detection occurred flag to true
            detection_occurred = True

    # Display the frame
    cv2.imshow('Frame', frame)

    # text to speech
    if detection_occurred:
        fp=str(value)+'Dinars'
        say(fp)        
        

    # Check for 'q' key press to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture device and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


In [7]:

!python3 detect.py --weights "/home/mhsn/Files/PFA/yolov5/best.pt" --source 0


Traceback (most recent call last):
  File "/home/mhsn/Files/PFA/detect.py", line 48, in <module>
    from models.common import DetectMultiBackend
ModuleNotFoundError: No module named 'models'


In [4]:
!cd /home/mhsn/Files/PFA/yolov5/
!pwd

/home/mhsn/Files/PFA
